In [259]:
from numpy import *
from pylab import *
from math import erf
import pandas as pd

In [260]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


D:\Program Files (x86)\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['linalg', 'fft', 'random', 'power']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [261]:
# constant cell

r = 0.12
sigma = 0.1
k = 100

N = 100
s_start = 0
s_end = 300
s = linspace(s_start, s_end, N + 1)
ds = abs(s_end - s_start)/N  # ds, uniform grid

M = 20
t_start = 0
t_end = 1  # T
time = 0
t = linspace(t_start, t_end, M + 1)
dt = abs(t_end - t_start)/M  # dt, uniform grid

In [262]:
# function cell for Euler's method

V = [0 for i in range(N + 1)]  # exact solution V(s, T-t)
W = [0 for i in range(N + 1)]  # numerical solution W(s, T-t)
s_second_partial = [0 for i in range(N + 1)]
s_first_partial = [0 for i in range(N + 1)]
time_error = [0 for i in range(M + 1)]

In [263]:
def aprx_func(W):
    for i in range(1,N):
        s_second_partial[i] = (W[i+1] - 2*W[i] + W[i-1])/ds**2 # midpoint formula for second order
        s_first_partial[i] = (W[i+1] - W[i-1])/(2*ds) # midpoint formula for first order
        
    s_second_partial[0] = 0; s_second_partial[N] = 0
    s_first_partial[0] = 0; s_first_partial[N] = 1
    
    return 1/2 * sigma**2 * (s*s)*s_second_partial + r*s*s_first_partial - r*float_(W)

def normal_dist(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def CDF(t, idx):
    arr = [0 for i in range(N + 1)]
    for i in range(1, N+1):
        if idx == 1:
            arr[i] = normal_dist((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
            #sym.erf((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
        else:
            arr[i] = normal_dist((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))
            #sym.erf((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))
            
    return arr

def exact_func(t):
    return s*float_(CDF(t,1)) - k*exp(-r*t)*float_(CDF(t,2))

In [264]:
# Finite difference method
# Euler's method for ODE

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
        V[i] = W[i]
    else:
        W[i] = 0
        V[i] = W[i]
        
#plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
#plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
#title('W(x) at time = ' + str(round(time, 2)))
#legend()
#axis((90, 150, 0, 45))
#pause(1)
e = 1
while time < t_end:
    time = time + dt
    
    W = W + dt*aprx_func(W) # answer fuction
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]  # initial condition: V(0,t) = 0, V(s_end,t) ~ s_end
    
    V = exact_func(time)
    time_error[M-e] = sum(abs(V-W))
    
    e=e+1
    #clf()
    #plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
    #plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
    #title('W(x) at time = ' + str(round(time, 2)))
    #axis((90, 150, 0, 45))
    #legend()
    #pause(0.001)
plot(t,time_error,linestyle='--',linewidth = 1.8, label='Euler')
legend()

In [256]:
time = 0

def aprx_(w, W):
    for i in range(1,N):
        s_second_partial[i] = (w[i+1] - 2*w[i] + w[i-1])/ds**2 # midpoint formula for second order
        s_first_partial[i] = (w[i+1] - w[i-1])/(2*ds) # midpoint formula for first order
        
    s_second_partial[0] = 0; s_second_partial[N] = 0
    s_first_partial[0] = 0; s_first_partial[N] = 1
    
    return 1/2 * sigma**2 * (s*s)*s_second_partial + r*s*s_first_partial - r*float_(W)


for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
        V[i] = W[i]
    else:
        W[i] = 0
        V[i] = W[i]
        
e = 1
while time < t_end:
    time = time + dt
    
    RK1 = aprx_(W,W)
    RK2 = aprx_(W,W + dt/2 * RK1)
    
    W = W + dt*RK2
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    time_error[M-e] = sum(abs(V-W))
    
    e=e+1
plot(t,time_error,linestyle='--',linewidth = 1.8, label='RK2')
legend()

In [215]:
clf()
N = 10
n = linspace(0,N,N+1)
dx = 1
V = exp(n)
W = [0 for i in range(N+1)]
W[0] = 1
for i in range(N):
    RK1 = W[i]
    RK2 = W[i] + dx*RK1
    
    W[i+1] = W[i] + dx*(RK1/2 + RK2/2)
    
    
error = abs(V-W)
#plot(n,V,linestyle='--',linewidth = 1.8, label='exact')
plot(n,error,linestyle='--',linewidth = 1.8, label='nume')
legend()

In [216]:
N = 10
n = linspace(0,N,N+1)
dx = 1
V = exp(n)
W = [0 for i in range(N+1)]
W[0] = 1
for i in range(N):
    RK1 = W[i]
    RK2 = W[i] + dx*RK1
    RK3 = W[i] + dx/2*(RK1+RK2)/2
    
    W[i+1] = W[i] + dx/6 * (RK1 + RK2 + 4*RK3)
    
    
error = abs(V-W)
plot(n,error,linestyle='--',linewidth = 1.8, label='numedd')
legend()

In [227]:
N = 10
n = linspace(0,N,N+1)
dx = 1
V = exp(n)
W = [0 for i in range(N+1)]
W[0] = 1
for i in range(N):
    RK1 = W[i]
    RK2 = W[i] + dx/2*RK1
    RK3 = W[i] + dx/2*RK2
    RK4 = W[i] + dx*RK3
    
    W[i+1] = W[i] + dx/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    
    
error = abs(V-W)
plot(n,error,linestyle='--',linewidth = 1.8, label='numeddddd')
legend()

In [257]:
time = 0

for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
        V[i] = W[i]
    else:
        W[i] = 0
        V[i] = W[i]
        
e = 1
while time < t_end:
    time = time + dt
    
    RK1 = aprx_(W,W)
    RK2 = aprx_(W,W + dt * RK1)
    RK3 = aprx_(W + dt/2 * (RK1 + RK2)/2)
    
    W = W + dt/6 * (RK1 + RK2 + 4*RK3)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    time_error[M-e] = sum(abs(V-W))
    
    e=e+1
plot(t,time_error,linestyle='--',linewidth = 1.8, label='RK3')
legend()

In [258]:
time = 0

for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
        V[i] = W[i]
    else:
        W[i] = 0
        V[i] = W[i]
        
e = 1
while time < t_end:
    time = time + dt
    
    RK1 = aprx_(W,W)
    RK2 = aprx_(W,W + dt/2 * RK1)
    RK3 = aprx_(W,W + dt/2 * RK2)
    RK4 = aprx_(W,W + dt * RK3)
    
    W = W + dt/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)

    time_error[M-e] = sum(abs(V-W))
    
    e=e+1
plot(t,time_error,linestyle='--',linewidth = 1.8, label='RK4')
legend()